In [1]:
%matplotlib inline

import librosa
import matplotlib.pyplot as plt
import librosa.display
import numpy as np
import torch
import os
import glob
import pickle
import copy
import random
import time
import traceback
from multiprocessing import Process

# Make file index dict

In [7]:
train_list = glob.glob('/Lun0/zhiyong/SdSV_2020_deepmine/task2_train/wav/train/*')

In [8]:
train_list

['/Lun0/zhiyong/SdSV_2020_deepmine/task2_train/wav/train/trn_106344.wav',
 '/Lun0/zhiyong/SdSV_2020_deepmine/task2_train/wav/train/trn_151403.wav',
 '/Lun0/zhiyong/SdSV_2020_deepmine/task2_train/wav/train/trn_162270.wav',
 '/Lun0/zhiyong/SdSV_2020_deepmine/task2_train/wav/train/trn_150448.wav',
 '/Lun0/zhiyong/SdSV_2020_deepmine/task2_train/wav/train/trn_157718.wav',
 '/Lun0/zhiyong/SdSV_2020_deepmine/task2_train/wav/train/trn_104892.wav',
 '/Lun0/zhiyong/SdSV_2020_deepmine/task2_train/wav/train/trn_144989.wav',
 '/Lun0/zhiyong/SdSV_2020_deepmine/task2_train/wav/train/trn_147787.wav',
 '/Lun0/zhiyong/SdSV_2020_deepmine/task2_train/wav/train/trn_130559.wav',
 '/Lun0/zhiyong/SdSV_2020_deepmine/task2_train/wav/train/trn_101127.wav',
 '/Lun0/zhiyong/SdSV_2020_deepmine/task2_train/wav/train/trn_176958.wav',
 '/Lun0/zhiyong/SdSV_2020_deepmine/task2_train/wav/train/trn_108219.wav',
 '/Lun0/zhiyong/SdSV_2020_deepmine/task2_train/wav/train/trn_143663.wav',
 '/Lun0/zhiyong/SdSV_2020_deepmine/tas

In [6]:
OPT_INDEX = '/Lun0/zhiyong/sdsvc_t2_train'

data_dir = '/Lun0/zhiyong/SdSV_2020_deepmine/task2_train/wav/train'
train_label_dir = '/Lun0/zhiyong/SdSV_2020_deepmine/task2_enrollment/docs/train_labels.txt'
new_start_id = 7323 #vox with 0-7322


spk2id = os.path.join(OPT_INDEX, 'spk2id')
spk2utt_dict = os.path.join(OPT_INDEX, 'spk2utt_dict')
spk2utt_train_dict = os.path.join(OPT_INDEX, 'spk2utt_train_dict')

In [17]:
train_labels2utt = {}
with open(train_label_dir, 'r') as f:
    for count, line in enumerate(f):
        if count == 0:
            continue
        line = line[:-1]
        utt, label = line.split(' ')
        if label not in train_labels2utt.keys():
            train_labels2utt[label] = [data_dir+'/'+utt+'.wav']
        else:
            train_labels2utt[label].append(data_dir+'/'+utt+'.wav')

In [18]:
len(train_labels2utt)

588

In [21]:
count = 0
for i in train_labels2utt:
    count += len(train_labels2utt[i])
print(count)

85764


In [22]:
spk2utt = {}

with open(spk2id, 'w') as f:
    pass


for count, i in enumerate(train_labels2utt):
    with open(spk2id, 'a') as f:
        f.write(str(count+new_start_id)+','+i+'\n')
    spk2utt[count+new_start_id] = train_labels2utt[i]

In [23]:
len(spk2utt)

588

In [26]:
spk2utt.keys()

dict_keys([7323, 7324, 7325, 7326, 7327, 7328, 7329, 7330, 7331, 7332, 7333, 7334, 7335, 7336, 7337, 7338, 7339, 7340, 7341, 7342, 7343, 7344, 7345, 7346, 7347, 7348, 7349, 7350, 7351, 7352, 7353, 7354, 7355, 7356, 7357, 7358, 7359, 7360, 7361, 7362, 7363, 7364, 7365, 7366, 7367, 7368, 7369, 7370, 7371, 7372, 7373, 7374, 7375, 7376, 7377, 7378, 7379, 7380, 7381, 7382, 7383, 7384, 7385, 7386, 7387, 7388, 7389, 7390, 7391, 7392, 7393, 7394, 7395, 7396, 7397, 7398, 7399, 7400, 7401, 7402, 7403, 7404, 7405, 7406, 7407, 7408, 7409, 7410, 7411, 7412, 7413, 7414, 7415, 7416, 7417, 7418, 7419, 7420, 7421, 7422, 7423, 7424, 7425, 7426, 7427, 7428, 7429, 7430, 7431, 7432, 7433, 7434, 7435, 7436, 7437, 7438, 7439, 7440, 7441, 7442, 7443, 7444, 7445, 7446, 7447, 7448, 7449, 7450, 7451, 7452, 7453, 7454, 7455, 7456, 7457, 7458, 7459, 7460, 7461, 7462, 7463, 7464, 7465, 7466, 7467, 7468, 7469, 7470, 7471, 7472, 7473, 7474, 7475, 7476, 7477, 7478, 7479, 7480, 7481, 7482, 7483, 7484, 7485, 7486, 7487,

In [27]:
count = 0
count_list = []
for i in spk2utt:
    count += len(spk2utt[i])
    count_list.append(len(spk2utt[i]))
print(count)

85764


In [28]:
with open(spk2utt_dict, 'wb') as handle:
    pickle.dump(spk2utt, handle)

with open(spk2utt_dict, 'rb') as handle:
    spk2utt = pickle.load(handle)
spk2utt_train = spk2utt

spk2utt_train = copy.deepcopy(spk2utt)

In [29]:
count = 0
count_list = []
for i in spk2utt_train:
    count += len(spk2utt_train[i])
    count_list.append(len(spk2utt[i]))
print(count)

85764


In [32]:
with open(spk2utt_train_dict, 'wb') as handle:
    pickle.dump(spk2utt_train, handle)

In [33]:
spk2utt_train_len = os.path.join(OPT_INDEX, 'spk2utt_train_len')

spk2utt_train_len_dict = {}

In [35]:
with open(spk2utt_train_dict, 'rb') as handle:
    spk2utt_train = pickle.load(handle)

for i in spk2utt_train:
    for utt in spk2utt_train[i]:
        spk2utt_train_len_dict[utt] = librosa.core.load(utt, sr=16000)[0].shape[0]

with open(spk2utt_train_len, 'wb') as handle:
    pickle.dump(spk2utt_train_len_dict, handle)

# Iterable Dataset & Make train data

In [ ]:
OPT_INDEX_1 = '/Lun0/zhiyong/dataset'
data_dir_dict = {}

# train
data_dir_dict['spk2utt_train_dict'] = os.path.join(OPT_INDEX, 'spk2utt_train_dict')
data_dir_dict['music_dict'] = os.path.join(OPT_INDEX_1, 'music_dict')
data_dir_dict['noise_dict'] = os.path.join(OPT_INDEX_1, 'noise_dict')
data_dir_dict['babble_dict'] = os.path.join(OPT_INDEX_1, 'babble_dict')
data_dir_dict['rir_dict'] = os.path.join(OPT_INDEX_1, 'rir_dict')

In [ ]:
data_len_dict = {}

data_len_dict['music_len'] = os.path.join(OPT_INDEX_1, 'music_len')
data_len_dict['noise_len'] = os.path.join(OPT_INDEX_1, 'noise_len')
data_len_dict['babble_len'] = os.path.join(OPT_INDEX_1, 'babble_len')
data_len_dict['spk2utt_train_len'] = os.path.join(OPT_INDEX, 'spk2utt_train_len')
# data_len_dict['spk2utt_val_len'] = os.path.join(OPT_INDEX, 'spk2utt_val_len')

In [ ]:
config = {}

config['sr'] = 16000
config['repeats'] = 150
config['batch_size'] = 1
config['extended_prefectch'] = 1.0

def iter_data_preload(dataset, i):
    dataset.get_random_list()
    try:
        start_time = time.time()
        for count, (data, label) in enumerate(dataset):
            with open('/Lun0/zhiyong/sdsvc_t2_train/train_data_sparse/'+str(i)+str('_')+str(count), 'wb') as handle:
                pickle.dump((data.astype(np.float16), label.astype(np.int16)), handle)
#             print(time.time()-start_time)
#             with open('/Lun0/zhiyong/sdsvc_t2_train/tmp_data_errlog_1', 'a') as f:
#                 f.write(str(time.time()-start_time)+'\n')
            start_time = time.time()
            if ((count+1)%10000) == 0:
                print((count+1)//10000)
    except Exception:
        with open('/Lun0/zhiyong/sdsvc_t2_train/tmp_data_errlog_1', 'a') as f:
            traceback.print_exc(file=f)
        raise Exception

dataset = VoxIterableDataset(data_dir_dict, data_len_dict, config)
# dataset.noise_data_preload2mem()
dataset.noise_data_preload()

processes = [Process(target = iter_data_preload, args = (dataset, i)) for i in range(10)]
start_time = time.time()
[p.start() for p in processes]
joined = [p.join() for p in processes]
print(time.time()-start_time)

# Make train index

In [7]:
588*1500

882000

In [8]:
# OPT_INDEX = '/Lun0/zhiyong/dataset'
train_data_dir = os.path.join(OPT_INDEX, 'train_data_sparse')
expected_len = 882000
workers = 10
single_worker_len = int(expected_len / workers)
output = os.path.join(OPT_INDEX, 'train_data_sparse.csv')

In [9]:
assert expected_len == len(glob.glob(train_data_dir+'/*'))

In [10]:
count = 0
with open(output, 'w') as f:
    for i in range(workers):
        for j in range(single_worker_len):
            path = os.path.join(train_data_dir, str(i)+'_'+str(j))
            assert os.path.isfile(path)
            f.write(path+'\n')
            count += 1

In [11]:
count

882000

# Make train data 488

In [7]:
OPT_INDEX = '/Lun0/zhiyong/sdsvc_t2_train'
OPT_INDEX_1 = '/Lun0/zhiyong/dataset'
spk2utt_train_dict = os.path.join(OPT_INDEX, 'spk2utt_train_dict')

In [8]:
with open(spk2utt_train_dict, 'rb') as handle:
    spk2utt_train = pickle.load(handle)

In [9]:
spk2utt_train_488 = {}
for count, i in enumerate(spk2utt_train):
    if count < 488:
        spk2utt_train_488[i] = spk2utt_train[i]

In [11]:
len(spk2utt_train_488.keys())

488

In [13]:
spk2utt_train_488[7323][0]

'/Lun0/zhiyong/SdSV_2020_deepmine/task2_train/wav/train/trn_101064.wav'

In [14]:
spk2utt_val_488 = {}
for i in spk2utt_train_488:
    spk2utt_val_488[i] = [spk2utt_train_488[i][0]]
    spk2utt_train_488[i].pop(0)

In [15]:
spk2utt_train_488[7323][0]

'/Lun0/zhiyong/SdSV_2020_deepmine/task2_train/wav/train/trn_101065.wav'

In [16]:
spk2utt_val_488[7323]

['/Lun0/zhiyong/SdSV_2020_deepmine/task2_train/wav/train/trn_101064.wav']

In [18]:
with open(os.path.join(OPT_INDEX, 'spk2utt_train_488'), 'wb') as handle:
    pickle.dump(spk2utt_train_488, handle)
with open(os.path.join(OPT_INDEX, 'spk2utt_val_488'), 'wb') as handle:
    pickle.dump(spk2utt_val_488, handle)

In [19]:
with open(os.path.join(OPT_INDEX, 'spk2utt_train_488'), 'rb') as handle:
    spk2utt_train_488 = pickle.load(handle)
with open(os.path.join(OPT_INDEX, 'spk2utt_val_488'), 'rb') as handle:
    spk2utt_val_488 = pickle.load(handle)

In [20]:
data_dir_dict = {}

# train
data_dir_dict['spk2utt_train_dict'] = os.path.join(OPT_INDEX, 'spk2utt_train_488')
data_dir_dict['music_dict'] = os.path.join(OPT_INDEX_1, 'music_dict')
data_dir_dict['noise_dict'] = os.path.join(OPT_INDEX_1, 'noise_dict')
data_dir_dict['babble_dict'] = os.path.join(OPT_INDEX_1, 'babble_dict')
data_dir_dict['rir_dict'] = os.path.join(OPT_INDEX_1, 'rir_dict')

In [21]:
data_len_dict = {}

data_len_dict['music_len'] = os.path.join(OPT_INDEX_1, 'music_len')
data_len_dict['noise_len'] = os.path.join(OPT_INDEX_1, 'noise_len')
data_len_dict['babble_len'] = os.path.join(OPT_INDEX_1, 'babble_len')
data_len_dict['spk2utt_train_len'] = os.path.join(OPT_INDEX, 'spk2utt_train_len')
# data_len_dict['spk2utt_val_len'] = os.path.join(OPT_INDEX, 'spk2utt_val_len')

In [22]:
config = {}

config['sr'] = 16000
config['repeats'] = 150
config['batch_size'] = 1
config['extended_prefectch'] = 1.0
config['out_dir'] = '/Lun0/zhiyong/sdsvc_t2_train/train_data_488_full/'
config['err_log'] = '/Lun0/zhiyong/sdsvc_t2_train/tmp_data_errlog_1'

def iter_data_preload(dataset, i):
    dataset.get_random_list()
    try:
        start_time = time.time()
        for count, (data, label) in enumerate(dataset):
            with open(config['out_dir']+str(i)+str('_')+str(count), 'wb') as handle:
                pickle.dump((data.astype(np.float16), label.astype(np.int16)), handle)
#             print(time.time()-start_time)
#             with open('/Lun0/zhiyong/sdsvc_t2_train/tmp_data_errlog_1', 'a') as f:
#                 f.write(str(time.time()-start_time)+'\n')
            start_time = time.time()
            if ((count+1)%10000) == 0:
                print((count+1)//10000)
    except Exception:
        with open(config['err_log'], 'a') as f:
            traceback.print_exc(file=f)
        raise Exception

dataset = VoxIterableDataset(data_dir_dict, data_len_dict, config)
# dataset.noise_data_preload2mem()
dataset.noise_data_preload()

processes = [Process(target = iter_data_preload, args = (dataset, i)) for i in range(10)]
start_time = time.time()
[p.start() for p in processes]
joined = [p.join() for p in processes]
print(time.time()-start_time)

preloading music_dict
100
200
300
400
500
600
preloading noise_dict
100
200
300
400
500
600
700
800
900
preloading babble_dict
100
200
300
400
1
1
1
1
1
1
1
1
1
1
2
2
2
2
2
2
2
2
2
2
3
3
3
3
3
3
3
3
3
3
4
4
4
4
4
4
4
4
4
4
5
5
5
5
5
5
5
5
5
5
6
6
6
6
6
6
6
6
6
6
7
7
7
7
7
7
7
7
7
7
13833.798538446426


# Make train index (Need only once)

In [30]:
OPT_INDEX = '/Lun0/zhiyong/sdsvc_t2_train'
train_data_dir = os.path.join(OPT_INDEX, 'train_data_488_full')
expected_len = 488 * 1500
workers = 10
single_worker_len = int(expected_len / workers)
output = os.path.join(OPT_INDEX, 'train_data_488_full.csv')

In [31]:
assert expected_len == len(glob.glob(train_data_dir+'/*'))

In [32]:
count = 0
with open(output, 'w') as f:
    for i in range(workers):
        for j in range(single_worker_len):
            path = os.path.join(train_data_dir, str(i)+'_'+str(j))
            assert os.path.isfile(path)
            f.write(path+'\n')
            count += 1

In [33]:
count

732000

# Make Val 488

In [23]:
OPT_INDEX = '/Lun0/zhiyong/sdsvc_t2_train'
OPT_INDEX_1 = '/Lun0/zhiyong/dataset'
data_dir_dict = {}

# val
data_dir_dict['spk2utt_train_dict'] = os.path.join(OPT_INDEX, 'spk2utt_val_488')
data_dir_dict['music_dict'] = os.path.join(OPT_INDEX_1, 'music_dict')
data_dir_dict['noise_dict'] = os.path.join(OPT_INDEX_1, 'noise_dict')
data_dir_dict['babble_dict'] = os.path.join(OPT_INDEX_1, 'babble_dict')
data_dir_dict['rir_dict'] = os.path.join(OPT_INDEX_1, 'rir_dict')

In [24]:
data_len_dict = {}


data_len_dict['music_len'] = os.path.join(OPT_INDEX_1, 'music_len')
data_len_dict['noise_len'] = os.path.join(OPT_INDEX_1, 'noise_len')
data_len_dict['babble_len'] = os.path.join(OPT_INDEX_1, 'babble_len')

data_len_dict['spk2utt_train_len'] = os.path.join(OPT_INDEX, 'spk2utt_train_len')

In [25]:
config = {}

config['sr'] = 16000
config['repeats'] = 4
config['batch_size'] = 1
config['extended_prefectch'] = 1.0
config['out_dir'] = '/Lun0/zhiyong/sdsvc_t2_train/val_data_488_full/'

def iter_data_preload(dataset, i):
    dataset.get_random_list()
    start_time = time.time()
    for count, (data, label) in enumerate(dataset):
        with open(config['out_dir']+str(i)+'_'+str(count), 'wb') as handle:
            pickle.dump((data.astype(np.float16), label.astype(np.int16)), handle)
        print(time.time()-start_time)
        start_time = time.time()

dataset = VoxIterableDataset(data_dir_dict, data_len_dict, config)
dataset.noise_data_preload()

processes = [Process(target = iter_data_preload, args = (dataset, i)) for i in range(1)]
start_time = time.time()
[p.start() for p in processes]
joined = [p.join() for p in processes]
print(time.time()-start_time)

preloading music_dict
100
200
300
400
500
600
preloading noise_dict
100
200
300
400
500
600
700
800
900
preloading babble_dict
100
200
300
400
0.18920254707336426
0.04179739952087402
0.05078887939453125
0.03654599189758301
0.031679391860961914
0.9931473731994629
0.031255483627319336
0.21308636665344238
0.05376267433166504
0.043622732162475586
0.2770848274230957
0.04679751396179199
0.02392721176147461
0.5604076385498047
0.046415090560913086
0.9926259517669678
0.8590197563171387
0.020342350006103516
0.01127767562866211
0.05234241485595703
0.02132439613342285
0.17650270462036133
0.01625537872314453
0.24042057991027832
0.9069364070892334
0.04106402397155762
0.037168264389038086
0.0737922191619873
0.0383603572845459
0.9920742511749268
0.9303028583526611
0.03661346435546875
0.06834745407104492
0.039671897888183594
0.9975531101226807
0.03549695014953613
0.048854827880859375
0.698218822479248
0.47563886642456055
0.07523083686828613
0.0677328109741211
0.06024169921875
0.040380001068115234
0.055

0.016440153121948242
0.4214518070220947
0.024126052856445312
0.04065299034118652
0.029927730560302734
0.07802772521972656
0.033354759216308594
0.03300189971923828
0.030092954635620117
0.047765493392944336
0.041924476623535156
0.017794132232666016
0.055124521255493164
0.04638171195983887
0.03464794158935547
0.03383517265319824
0.0457768440246582
0.07585549354553223
0.033089637756347656
0.011889934539794922
0.01918625831604004
0.07658505439758301
0.12560701370239258
0.04774117469787598
0.011927366256713867
0.0398256778717041
0.036208152770996094
0.42707180976867676
0.022863388061523438
0.01890277862548828
0.03726458549499512
0.038051605224609375
0.03371930122375488
0.03318142890930176
0.04090714454650879
0.0184934139251709
0.22351503372192383
0.04820585250854492
0.46326422691345215
0.037863969802856445
0.05198407173156738
0.04704117774963379
0.013554811477661133
0.036071062088012695
0.01925373077392578
0.029708385467529297
0.031438589096069336
0.028131484985351562
0.04205060005187988
0.0

0.014899730682373047
0.012462377548217773
0.018228769302368164
0.05298113822937012
0.024797677993774414
0.024916887283325195
0.03375673294067383
0.12879276275634766
0.03181052207946777
0.09809350967407227
0.014204978942871094
0.0276949405670166
0.014633655548095703
0.01964545249938965
0.025404691696166992
0.027896404266357422
0.06042957305908203
0.022794246673583984
0.01412057876586914
0.03431391716003418
0.022568225860595703
0.01249837875366211
0.013450384140014648
0.015724897384643555
0.012051820755004883
0.01409912109375
0.020532846450805664
0.031119585037231445
0.04955434799194336
0.025417566299438477
1.100656270980835
1.0773513317108154
0.04588937759399414
0.014563560485839844
0.08310604095458984
0.029035329818725586
0.01532292366027832
0.02262735366821289
0.016940593719482422
0.0395808219909668
0.028805017471313477
0.05760765075683594
0.014108896255493164
0.013673543930053711
0.017536163330078125
0.0331575870513916
0.017797470092773438
0.038071393966674805
0.023736953735351562
0.

0.01841902732849121
0.01239919662475586
0.02441859245300293
0.02061915397644043
0.022791385650634766
0.032335758209228516
0.013811349868774414
0.013450860977172852
0.05828452110290527
0.045540571212768555
0.02543163299560547
0.03019857406616211
0.029105424880981445
0.017002582550048828
0.022793292999267578
0.01914501190185547
0.019004106521606445
0.013907670974731445
0.027827024459838867
0.014431238174438477
0.01463627815246582
0.027680158615112305
0.024906635284423828
0.03307342529296875
0.014402151107788086
0.018709659576416016
0.01494598388671875
0.029485464096069336
0.020574092864990234
0.025258541107177734
0.011102437973022461
0.02727341651916504
0.026272058486938477
0.014544963836669922
0.036837100982666016
0.013618946075439453
0.019327402114868164
0.014017105102539062
0.013313531875610352
0.014890670776367188
0.03747105598449707
0.07046175003051758
0.03585648536682129
0.02794170379638672
0.05053067207336426
0.02531576156616211
0.03756356239318848
0.0180971622467041
0.02284550666

0.014043092727661133
0.016017913818359375
0.014398574829101562
0.015833616256713867
0.03579545021057129
0.01819157600402832
0.018347501754760742
0.01734304428100586
0.014032125473022461
0.012378454208374023
0.012676477432250977
0.019040584564208984
0.023319482803344727
0.021361589431762695
0.019000768661499023
0.03002619743347168
0.013682126998901367
0.013567209243774414
0.013222455978393555
0.02504730224609375
0.0406644344329834
0.027934551239013672
0.02447676658630371
0.015263795852661133
0.02462935447692871
0.031087875366210938
0.0227963924407959
0.022266626358032227
0.026021957397460938
0.014667272567749023
0.012430667877197266
0.02319025993347168
0.01469111442565918
0.02502751350402832
0.012206315994262695
0.013593912124633789
0.014236211776733398
0.01797318458557129
0.014408111572265625
0.014303922653198242
0.02317214012145996
0.014142751693725586
0.015798091888427734
0.030223369598388672
0.014583826065063477
0.014667510986328125
0.01646256446838379
0.03225064277648926
0.01589107

# Make validation index (Need only once)

In [26]:
OPT_INDEX = '/Lun0/zhiyong/sdsvc_t2_train'
train_data_dir = os.path.join(OPT_INDEX, 'val_data_488_full')
expected_len = 488 * 4
workers = 1
single_worker_len = int(expected_len / workers)
output = os.path.join(OPT_INDEX, 'val_data_488_full.csv')

In [27]:
assert expected_len == len(glob.glob(train_data_dir+'/*'))

In [28]:
count = 0
with open(output, 'w') as f:
    for i in range(workers):
        for j in range(single_worker_len):
            path = os.path.join(train_data_dir, str(i)+'_'+str(j))
            assert os.path.isfile(path)
            f.write(path+'\n')
            count += 1

In [29]:
count

1952